In [346]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import constants
from scipy import stats
from scipy.optimize import leastsq

In [347]:
k = constants.value('Boltzmann constant in eV/K')
e = np.exp(1)

In [348]:
# Read in data
GRDF = pd.read_csv("../Data/GrowthRespPhotoData_new.csv",
                   usecols = ["FinalID",
                              "StandardisedTraitName",
                              "StandardisedTraitDef",
                              "StandardisedTraitValue",
                              "StandardisedTraitUnit",
                              "AmbientTemp",
                              "AmbientTempUnit",
                              "ConTemp",
                              "ConTempUnit",
                              "ResTemp",
                              "ResTempUnit"],
                   low_memory = False)

In [349]:
# create NewID based on FinalID
GRDF["NewID"] = GRDF.FinalID.astype("category").cat.codes

In [350]:
# get rid of 0s -ves and NAs from data (only +ves)
GRDF = GRDF.loc[GRDF.index[GRDF["StandardisedTraitValue"] > 0]]

In [351]:
# return Relevent Temp
def f(group):
    if group.ConTemp.isnull().any():
        if group.ResTemp.isnull().any():
            group["UsedTemp"]     = group.AmbientTemp
            group["UsedTempUnit"] = group.AmbientTempUnit
            group["UsedTempType"] = "AmbientTemp"
            return group
        else:
            group["UsedTemp"]     = group.ResTemp
            group["UsedTempUnit"] = group.ResTempUnit
            group["UsedTempType"] = "ResTemp"
            return group
    else:
        group["UsedTemp"]     = group.ConTemp
        group["UsedTempUnit"] = group.ConTempUnit
        group["UsedTempType"] = "ConTemp"
        return group

GRDF = GRDF.groupby("NewID").apply(f)

In [352]:
# Convert to Kelvin
GRDF["UsedTempK"] = GRDF.UsedTemp + 273.15

In [353]:
# Sort by ID then temperature
GRDF = GRDF.sort_values(['NewID', 'UsedTempK'])

In [354]:
# Removes groups where all temp values are the same
def f(group):
    if len(group.UsedTemp.unique()) == 1:
        return False
    else:
        return True
    
GRDF = GRDF.groupby("NewID").filter(f)

In [355]:
# Removes groups where all trait values are the same
def f(group):
    if len(group.StandardisedTraitValue.unique()) == 1:
        return False
    else:
        return True
    
GRDF = GRDF.groupby("NewID").filter(f)

In [356]:
# reomve first point if its much higher (change to difference between points?)
def f(group):
    try:
        if group.reset_index().StandardisedTraitValue[0] > 3*group.reset_index().StandardisedTraitValue[1]:
            return group.reset_index()[1:]
        else:
            return group.reset_index()
    except KeyError:
        return group.reset_index()
    
GRDF = GRDF.groupby("NewID").apply(f)

In [357]:
GRDF = GRDF.loc[ : ,("FinalID",
                     "StandardisedTraitName",
                     "StandardisedTraitDef",
                     "StandardisedTraitValue",
                     "StandardisedTraitUnit",
                     "UsedTemp",
                     "UsedTempType",
                     "UsedTempK")]

In [358]:
# Remove groups with fewer than five
GRDF = GRDF.groupby("NewID").filter(lambda x: len(x) > 5)

In [359]:
# logged trait value
GRDF["STVlogged"] = np.log(GRDF.StandardisedTraitValue)

In [360]:
# 1/kT
GRDF["adjTemp"] = 1/(GRDF.UsedTempK*k)

In [363]:
GRDF.to_csv("../Results/sorted_data.csv")